In [1]:
from astropy.io import fits
import pandas as pd
import numpy as np
from glob import glob
import os
from copy import deepcopy
from joblib import Parallel, delayed

In [2]:
darks = pd.read_csv('../pouakai/cal_lists/dark_list.csv')

In [6]:
darks['jd'].values.astype(int)

array([2459683, 2459683, 2459686, 2459683, 2459687, 2459605, 2459686,
       2459825, 2459821, 2459687, 2459683, 2459683, 2459821, 2459825,
       2459821, 2459611, 2459611, 2459686, 2459831, 2459687, 2459822,
       2459683, 2459825, 2459687, 2459825, 2459687, 2459686, 2459605,
       2459686, 2459683, 2459822, 2459686, 2459687, 2459686, 2459686,
       2459825, 2459825, 2459606, 2459822, 2459821, 2459606, 2459605,
       2459683, 2459684, 2459687, 2459825, 2459683, 2459686, 2459825,
       2459687, 2459687, 2459683, 2459825, 2459831, 2459825, 2459604,
       2459683, 2459686, 2459686, 2459617, 2459825, 2459831, 2459687,
       2459821, 2459605, 2459686, 2459687, 2459608, 2459825, 2459686,
       2459825, 2459605, 2459822, 2459686, 2459684, 2459608, 2459617,
       2459821, 2459608, 2459683, 2459821, 2459821, 2459822, 2459686,
       2459687, 2459605, 2459686, 2459604, 2459604, 2459825, 2459683,
       2459821, 2459606, 2459606, 2459683, 2459686, 2459825, 2459604,
       2459821, 2459

In [22]:
def assign_master_name(darks):
    date = darks['date'].values
    d = deepcopy(date)
    for i in range(len(d)):
        d[i] =  d[i].split('T')[0].replace('-','') 
    exptime = darks['exptime'].values.astype(int).astype(str)
    master_name = 'dark_' + d + '_' + exptime +'s'
    return master_name

In [23]:
master_name = assign_master_name(darks)

In [27]:
names = list(set(master_name))

In [32]:
darks.iloc[inds[0]]

,name,telescope,exptime,jd,date,filename,note
37,dark-0003_60,B&C,60.0,2.459607e+06,2022-01-27T05:01:32.91,/home/phys/astro8/MJArchive/octans/20220127/da...,NaN
93,dark-0002_60,B&C,60.0,2.459607e+06,2022-01-27T04:50:17.42,/home/phys/astro8/MJArchive/octans/20220127/da...,NaN
123,dark-0005_60,B&C,60.0,2.459607e+06,2022-01-27T05:24:03.94,/home/phys/astro8/MJArchive/octans/20220127/da...,NaN
124,dark-0004_60,B&C,60.0,2.459607e+06,2022-01-27T05:12:48.42,/home/phys/astro8/MJArchive/octans/20220127/da...,NaN
568,dark-0001_60,B&C,60.0,2.459607e+06,2022-01-27T04:39:01.92,/home/phys/astro8/MJArchive/octans/20220127/da...,NaN


In [ ]:
def make_master_darks(save_location = '/home/phys/astronomy/rri38/fli/data/master/dark/',num_cores=25,verbose=False):
    # make save_location an environment variable
    dark_list = pd.read_csv('cal_lists/dark_list.csv')
    masters = pd.read_csv('cal_lists/master_dark_list.csv')
    master_name = assign_master_name(darks)
    all_names = set(master_name)
    master_names = set(masters['name'].values)

    new = all_names ^ master_names
    new = list(new)
    new.sort(reverse=True)
    print('sorted')
    inds = []
    for i in range(len(new)):
        narr = deepcopy(all_names)
        narr[:] = names[i]
        ind = narr == all_names
        inds += [ind]
    indexer = np.arange(len(new),dtype=int)
    entries = Parallel(n_jobs=num_cores)(delayed(dark_processing)(ind,new,dark_list,save_location,verbose) for ind in inds)
    for entry in entries:
        masters = masters.append(entry,ignore_index=True)
    masters.to_csv('cal_lists/master_dark_list.csv',index=False)

In [ ]:
def dark_processing(index,new,dark_list,save_location,verbose):
    entry = {}
    name = new[index][0]
    all_new = dark_list.iloc[index]
    files = all_new['filename'].values
    master = []
    for file in files:
        hdu = fits.open(file)[0]
        header = hdu.header
        data = hdu.data
        master += [data]
    master = np.array(master)
    #print('made array')
    if verbose:
        print('Used ',len(master),' images in median')
    m = np.nanmedian(master,axis=0)
    std = np.nanstd(master,axis=0)
    time = np.nanmean(all_new['jd'])
    #print('calc mean')
    header['JDSTART'] = time 
    header['MASTER'] = True
    phdu = fits.PrimaryHDU(data = m, header = header)
    ehdu = fits.ImageHDU(data = std, header = header)
    hdul = fits.HDUList([phdu, ehdu])

    save_name = save_location + name + '.fits'
    print('saving')
    hdul.writeto(save_name,overwrite=True)
    compress = 'gzip -f ' + save_name
    os.system(compress)
    print('saved')
    entry['name'] = name

    entry['chip'] = header['CHIP']
    entry['exptime'] = header['EXPTIME']
    entry['jd'] = time
    entry['date'] = header['DATE-OBS']
    entry['nimages'] = len(master)
    entry['filename'] = save_name + '.gz'
    if len(master) < 3:
        note = 'bad'
    else:
        note = 'good'
    entry['note'] = note
    if verbose:
        print('Done ', name)
    entry = pd.DataFrame([entry])
		
    return entry